# Interacting with a Pinecone index

## Creating an index

In [ ]:
import os
import time
import pandas as pd
from pinecone import Pinecone
from dotenv import load_dotenv
from pinecone import ServerlessSpec

load_dotenv()

In [ ]:
# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone_api_key = os.getenv("PINECONE_API_KEY")
pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [ ]:
index_name = "interacting-with-the-index"

In [ ]:
# Delete index if exists
if index_name in pc.list_indexes().names():
    pc.delete_index(index_name)

In [ ]:
# Create index
pc.create_index(
    name=index_name, 
    dimension=2, 
    metric="euclidean",
    spec=spec
)

In [ ]:
# wait for index to be ready before connecting
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

# Connect to the index
index = pc.Index(index_name)

In [ ]:

df = pd.DataFrame()
df["id"] = ["A", "B", "C", "D", "E"]
df["vector"] = [[1., 1.], [2., 2.], [3., 3.], [4., 4.], [5., 5.]]
df

In [ ]:
# Upsert the vectors
AB_df = df[:2]
index.upsert(vectors=zip(AB_df.id, AB_df.vector))

## Fetch vectors

In [ ]:
fetch_results=index.fetch(ids=["A", "B"])
fetch_results

## Query top-k vectors

In [ ]:
result=index.query(vector=[1.1,1.1],top_k=2)
result

## Update vectors by ID

In [ ]:
# Fetch current vectors by ID
fetch_result = index.fetch(ids=["A"])
fetch_result

In [ ]:
# Update vectors by ID
index.upsert(vectors=[("A",[0.1, 0.1])])

In [ ]:
# Fetch vector by the same ID again
fetch_result = index.fetch(ids=["A"])
fetch_result

## Delete vectors by ID

In [ ]:
# Delete vectors by ID
index.delete(ids=["A"])

In [ ]:
# Deleted vectors are empty
fetch_results = index.fetch(ids=["A", "B"])
fetch_results

## Get index statistics

In [ ]:
# Index statistics
index.describe_index_stats()

## Delete the index


In [18]:
# delete the index
pc.delete_index(index_name)